In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import foolbox as fb
from foolbox.attacks import LinfPGD

In [7]:
def save_image(img, path):
    img = img.squeeze(0)  # remove batch dimension if present
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imsave(path, np.transpose(npimg, (1, 2, 0)))

# load CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

# create directories for ImageFolder structure
output_dir = 'cifar10_adversarial_epsilon_02'
original_dir = os.path.join(output_dir, 'original')
adversarial_dir = os.path.join(output_dir, 'adversarial')

os.makedirs(original_dir, exist_ok=True)
os.makedirs(adversarial_dir, exist_ok=True)

# create subdirectories for each class
for class_idx in range(10):
    os.makedirs(os.path.join(original_dir, str(class_idx)), exist_ok=True)
    os.makedirs(os.path.join(adversarial_dir, str(class_idx)), exist_ok=True)

# define a simple model (using pretrained ResNet18)
model = torchvision.models.resnet18(pretrained=True)
model.eval()

# create Foolbox model
fmodel = fb.PyTorchModel(model, bounds=(-1, 1))

# define adversarial attack
attack = LinfPGD()
epsilons = [0.3] # TODO: play around with this for exaperimens 

# process and save each image
counter = 0 
for idx, (images, labels) in enumerate(testloader):
    if counter ==500: # TODO: remove this, at the moment is only saving ~500 images 
        break 
    label = labels.item()  #get the class label
    
    # save original image
    original_path = os.path.join(original_dir, str(label), f"{idx}.png")
    save_image(images[0], original_path)
    
    # apply attack to generate adversarial image
    adversarials = attack(fmodel, images, labels, epsilons=epsilons)
    adversarial_image = adversarials[0]

    # ensure adversarial_image is a tensor
    if isinstance(adversarial_image, list):
        adversarial_image = adversarial_image[0]
    if isinstance(adversarial_image, torch.Tensor):
        adversarial_image = adversarial_image.cpu()
    
    # save adversarial image
    adversarial_path = os.path.join(adversarial_dir, str(label), f"{idx}.png")
    save_image(adversarial_image, adversarial_path)

    # print progress
    if idx % 100 == 0:
        print(f"Processed {idx}/{len(testloader)} images")
    counter +=1

print(idx," images saved in ImageFolder format")

Files already downloaded and verified


/Users/rosameliacarioni/miniconda3/envs/b_cos/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/rosameliacarioni/miniconda3/envs/b_cos/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processed 0/10000 images
Processed 100/10000 images
Processed 200/10000 images
Processed 300/10000 images
Processed 400/10000 images
All images saved in ImageFolder format.
